In [ ]:
from keras.preprocessing import image
from keras.applications import resnet50
%matplotlib inline
import os, math
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
#following added by JMS 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.core import Dense, Flatten
from keras.optimizers import SGD, Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import itertools
import matplotlib.pyplot as plt
from keras.models import load_model
import h5py

In [ ]:
def label_names(label_vector, classes):
    """
    Get label names from y vector.
    
    Args:
        label_vector : vector of ones and zeros representing y_value of a data instance
        classes : a list of all the possible classes whose order aligns with y_value
    """
    assert label_vector.size == len(classes)
    l = []
    for i,x in enumerate(label_vector):
        if x==1:
            l.append(classes[i])
    return l

In [ ]:
# Below taken from fastai
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None, classes=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(label_names(titles[i], classes), fontsize=8)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
# Code taken from stackoverflow post
def zca_whitening_matrix(X):
    """
    Function to compute ZCA whitening matrix (aka Mahalanobis whitening).
    INPUT:  X: [M x N] matrix.
        Rows: Variables
        Columns: Observations
    OUTPUT: ZCAMatrix: [M x M] matrix
    """
    # Covariance matrix [column-wise variables]: Sigma = (X-mu)' * (X-mu) / N
    sigma = np.cov(X, rowvar=True) # [M x M]
    # Singular Value Decomposition. X = U * np.diag(S) * V
    U,S,V = np.linalg.svd(sigma)
        # U: [M x M] eigenvectors of sigma.
        # S: [M x 1] eigenvalues of sigma.
        # V: [M x M] transpose of U
    # Whitening constant: prevents division by zero
    epsilon = 1e-5
    # ZCA Whitening matrix: U * Lambda * U'
    ZCAMatrix = np.dot(U, np.dot(np.diag(1.0/np.sqrt(S + epsilon)), U.T)) # [M x M]
    return ZCAMatrix

In [ ]:
def whiten_batch(X):
    X_norm = np.multiply(X, 1.0/255.0, casting="unsafe")
    flatx = np.reshape(X_norm, (-1, np.prod(X_norm.shape[-3:])))
    ZCAMatrix = zca_whitening_matrix(flatx)
    ZCAMatrix.shape
    xZCAMatrix = np.dot(ZCAMatrix, flatx) # project X onto the ZCAMatrix
    X_hat = np.reshape(xZCAMatrix, X.shape)
    return X_hat

<h2>Loading Data</h2>
<br>

In [ ]:
BATCH_SIZE = 32

In [ ]:
train_path = "data/train.h5"
valid_path = "data/validate.h5"
train_augment_path = "data/train_augment.h5"
valid_augment_path = "data/validate_augment.h5"

In [ ]:
with h5py.File(train_path, 'r') as hf:
    X_train, y_train = hf['images'][:], hf['labels'][:]

with h5py.File(valid_path, 'r') as hf:
    X_valid, y_valid = hf['images'][:], hf['labels'][:]
        

In [ ]:
use_augment = True

In [ ]:
# dataset also contains augmented data
if use_augment:
    with h5py.File(train_augment_path, 'r') as hf:
        X_train_aug, y_train_aug = hf['images'][:], hf['labels'][:]

    with h5py.File(valid_augment_path, 'r') as hf:
        X_valid_aug, y_valid_aug = hf['images'][:], hf['labels'][:]
    
    
    X_train = np.concatenate((X_train, X_train_aug), axis=0)
    y_train = np.concatenate((y_train, y_train_aug), axis=0)
    X_valid = np.concatenate((X_valid, X_valid_aug), axis=0)
    y_valid = np.concatenate((y_valid, y_valid_aug), axis=0)

In [ ]:
print(y_train.shape, y_train_aug.shape)

In [ ]:
datagen = ImageDataGenerator(featurewise_center=True,
                             featurewise_std_normalization=True,
                             rotation_range=20,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True)


In [ ]:
datagen.fit(X_train)

In [ ]:
classes = ['cat', 'dog', 'rabbit']

<h2>Creating/Loading Pre-Trained ResNet Model</h2>
<br>

In [ ]:
# Load Keras' ResNet50 model that was pre-trained against the ImageNet database
model = resnet50.ResNet50()

In [ ]:
#take off last layer
model.layers.pop()

#freeze layers
for layer in model.layers[:-2]:
    layer.trainable=False

#make last layer    
last = model.layers[-1].output
x = Dense(len(classes), activation="softmax")(last)

In [ ]:
#set training steps based on the # of images
num_train_samples = X_train.shape[0]
num_valid_samples = X_valid.shape[0]
# num_valid_samples = sum([len(files) for r, d, files in os.walk(validate_path)])

In [ ]:
num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)


<h2>Compile and Train model</h2>
<br>

In [ ]:
def testModel(X, y, model):
    predictions = model.predict(X, True)
    # for each image get the index of the class with max probability
    int_predictions = np.argmax(predictions, axis=1)
    int_labels = np.argmax(y, axis=1)
    
    count = 0
    for i in range(int_labels.size):
        if int_predictions[i] == int_labels[i]:
            count += 1
    return count/int_labels.size

In [ ]:
epochs = 80
learning_rate = 0.0001
decay_rate = learning_rate / epochs
momentum = 0.8
adam = Adam(lr=learning_rate, beta_1=0.8, beta_2=0.999, decay=decay_rate, epsilon=0.0000001)

In [ ]:
#finetune model 
finetuned_model = Model(model.input, x)
finetuned_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

best_loss = float("inf")

for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=BATCH_SIZE):
        X_batch_whitened = whiten_batch(X_batch)
        finetuned_model.fit(X_batch_whitened, y_batch)
        batches += 1
#         print("batch:", batches)
        if batches >= len(X_train) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    
    score = finetuned_model.evaluate(X_valid, y_valid, verbose=0)
    if score[0] < best_loss:
        print("Validation loss improved from", best_loss, "to", score[0])
        best_loss = score[0]
        finetuned_model.save('models/resnet50.h5')
        
    print('Valid loss:', score[0], 'Valid accuracy:', score[1])

finetuned_model.save('models/resnet50.h5')

In [ ]:
pred_probs = finetuned_model.predict(images, True)
# for each image get the index of the class with max probability
predictions = np.argmax(pred_probs, axis=1)

<h2>Confusion Matrix</h2>
<br>

In [ ]:
true_labels = np.argmax(labels, axis=1)

In [ ]:
cm = confusion_matrix(true_labels, predictions)

In [ ]:
# From Scikit Learn

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    fig = plt.gcf()
    fig.set_size_inches(16, 16)

In [ ]:
cm_plot_labels = classes
plot_confusion_matrix(cm, cm_plot_labels, title = 'Confusion Matrix')

<h2>Stacked Histogram</h2>
<br>

In [ ]:
def accuracy_by_label(true_labels, predicted_labels, label_names):
    correct, incorrect = [0]*len(label_names), [0]*len(label_names)
    for true, predict in zip(true_labels, predicted_labels):
        if true == predict:
            correct[true] += 1
        else:
            incorrect[true] += 1
    return correct, incorrect

In [ ]:
N = len(classes)
correct, incorrect = accuracy_by_label(true_labels, predictions, classes)
ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = plt.bar(ind, correct, width)
p2 = plt.bar(ind, incorrect, width,
             bottom=correct)

plt.ylabel('Number of Images')
plt.title('Correct Classification vs Incorrect Classification')
plt.xticks(ind, classes)
plt.yticks(np.arange(0, 100, 20))
plt.legend((p1[0], p2[0]), ('Correct', 'Incorrect'))
fig = plt.gcf()
fig.set_size_inches(28, 18)
plt.show()